# Disaster Tweets, Real or Not?

## Summary

### Problem Statement

*Think about the problem you are trying to solve as a business case. After you've built your model, who is going to use it and what will they do with it? Who are your customers and stakeholders and what is their need? The need is not the data they are trying to understand -- think of the need as a specific action they would like to take or a question they want to answer.*

*What is your model going to **do** in the real world? If you can answer this, and if you keep that answer in mind as you complete your work, you will have a much clearer view of where to go at each step.*

* Who are your customers?
* What is the problem?
* What solution do you propose?
* How will you know if your work is good?

*Also, think critacally here. If someone was paying you to do this work, what would they want you to build.*

### Work Plan

*Now that you have a clear idea of what you are trying to accomplish, you need to find a data source. Think of this as working through a maze forwards and backwards at the same time. At the start you have any number of data sets available to work with (and the whole Internet to search and scrape), and at the end is the hypothetical data set that would answer your question immediately if you had it.*

* What data, if you had it, would solve your problem right away?
* What data do you have access to?
* What additional data would be good to have?
* What data would be impossible to collect?
* What are the best proxies you can find for unavailable or impossible data?
* What are the legal or ethical issues you might run into if you were to try to collect all of the types of data you would like to work with?

*Of course you need to make a plan to turn your data into the solutions you need. Think of what type of problem this is, what models are commonly used for those types of problems, what types of data those models require and special considerations that may need to be made. Do your homework and find out what approaches other people are using on similar tasks.*

* What ML paradigm are you working in? (Classification, Regression, Clustering, etc)
* What models are commonly used in this task?
* What other solutions are being tried in this field?
* What special considerations need to be taken when dealing with these models? (i.e., imbalanced classes, text preprocessing, data leakage, etc)
* How will you know that your models work?
* How will you recognize and diagnose cases where the predictions are incorrect?

## Imports

*Keep all your imports in one place. This will make it much easier to see everything that you are using*

In [1]:
# Make sys.path mimic the location of the mvp.ipynb notebook
import sys
sys.path.append('..')

import numpy as np
import pandas as pd

# Source python scripts
import src.acquire
import src.prepare
import src.explore
import src.model

## Acquire the Data

### Data Pipeline

*This is where you describe the ETL (Extract, Transform, Load) process. Describe the actions you will need to take to obtain the data from where it is stored, and convert it into a clean format. Also be aware of how the available data is likely to change over time -- how often will it be updated, and how long will the available data be relevant?*

*After completing this step, be sure to edit the README data dictionary to include descriptions of where you obtained your data and what information it contains.*

### Create your Pipeline Tools

*Your work is going to be far less useful if anyone who wants to use it or build on it is going to have to put together a new data set from scratch. Automate everything that you possibly can. (And if you don't think it can be automated, ask your peers and check Google to see what's out there.)*

*The pipeline tools will connect to data sources, create local directories to store the data, download the data, clean it, reformat it, and store it. In the following step, "Run the Pipeline," you'll write a function that uses all of these tools to build your data set in one line of code.*

*Make sure these steps are reproducible by code. Put some thought into the directory structures and filepaths you are using to save your data, so it's easy to load files you need.*

In [2]:
def download_data():
    os.system("kaggle competitions download nlp-getting-started -f test.csv -p data/raw/")
    os.system("kaggle competitions download nlp-getting-started -f train.csv -p data/raw/")

def data_exists():
    return os.path.isfile("data/raw/test.csv") and os.path.isfile("data/raw/train.csv")

def run():
    print("Acquire: downloading raw data files...")

    if data_exists():
        print("Data already exists")
    else:
        download_data()
        print("Data acquired")

    print("Acquire: Completed!")

### Prove it Works

*Once your tools are built (and are working reliably!!), you'll be able to work much faster if they're out of sight. Collaborators and data scientists who build on your work will also be much more productive if you provide easy access to the data. So do yourself and your team a favor, and package your pipeline tools into a single function that can easily reproduce your working data set. This is what our run function in acquire is for.*

*Also a helpful thing to remember is to set this function up so that when you re-run the pipeline, it checks whether the files are already available locally. This will save you lots of time downloading large files.*

In [3]:
src.acquire.run()
df = pd.read_csv('data/raw/train.csv')

Acquire: downloading raw data files...
Data acquired
Acquire: Completed!


## Prepare the Data

*This is the important phase and it is seen as a chore by some, even though I personally enjoy the puzzle that this stage always provides. If you tend to think of data scrubbing as a drag on your workflow, it's well worth your time to explore some new pandas functions you're unfamiliar with and speed up your workflow. Try to learn one or two new tricks with each project you work on.*

*The primary scientific component to scrubbing data is going to be the thought process around deciding what to do with missing data. There are frequently no right or wrong answers for what to do with missing values, so be sure to record your thoughts, and share your process with peers or collaborators to get a sanity check.*

### Load (and Split) the Data

*You may already have your data loaded in this notebook after the Acquire stage has been run. But you probably don't want to have to re-run your data gathering pipeline every time you try a new idea for your model. Write some code here to load your cached data set so you won't have to start from scratch.*

*It is especially important to go back to a clean data set with each iteration that you want to test or add to your model. Your new results will not be reproducible if this iteration is developed on data that has been transformed in any way during the previous iteration.*

*Lastly if possible split the data at this stage. It is always best to split the data as early as possible so that you are making you decision of preparation on a subset of the whole dataset as the model you are building will be doing on future unseen data*

In [4]:
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [6]:
df.keyword.value_counts()

fatalities               45
deluge                   42
armageddon               42
harm                     41
sinking                  41
                         ..
forest%20fire            19
epicentre                12
threat                   11
inundation               10
radiation%20emergency     9
Name: keyword, Length: 221, dtype: int64

In [7]:
df.location.value_counts()

USA                             104
New York                         71
United States                    50
London                           45
Canada                           29
                               ... 
probably not home                 1
ChicagoRObotz                     1
Rockland County, NY               1
westwestwestwestwestwestwest      1
Lincoln, IL                       1
Name: location, Length: 3341, dtype: int64

In [8]:
df.isna().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [9]:
df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [10]:
# We need to fill all the NA data for the keyword and location columns

In [11]:
df.keyword.fillna('no_keyword', inplace = True)

In [12]:
df.location.fillna('no_location', inplace = True)

In [13]:
df.head()

,id,keyword,location,text,target
0,1,no_keyword,no_location,Our Deeds are the Reason of this #earthquake M...,1
1,4,no_keyword,no_location,Forest fire near La Ronge Sask. Canada,1
2,5,no_keyword,no_location,All residents asked to 'shelter in place' are ...,1
3,6,no_keyword,no_location,"13,000 people receive #wildfires evacuation or...",1
4,7,no_keyword,no_location,Just got sent this photo from Ruby #Alaska as ...,1


In [14]:
df.isna().sum()

id          0
keyword     0
location    0
text        0
target      0
dtype: int64

### Clean the Data

*Make sure that your data is in a format that your models can work with, and fix any problems that need attention. This may include formatting strings, removing duplicate records, and handling missing data.*

*There are multiple approaches to handling missing records, including removing them, inferring the values either statistically or from other insights, or finding other data sources to fill the gaps. Be sure to note down your thought process for whatever decision you make.*

In [4]:
# Write functions that prepare the data from your data/raw and store
# the prepared data in data/
# Once that is done move the functions to prepare.py

## Explore the Data

**If you haven't split the data yet, do it before you explore**

*Before you start exploring the data, write out your thought process about what you're looking for and what you expect to find. Take a minute to confirm that your plan actually makes sense.*

*Calculate summary statistics and plot some charts to give you an idea what types of useful relationships might be in your dataset. Use these insights to go back and download additional data or engineer new features if necessary. Not now though... remember we're still just trying to finish the MVP!*

### Explore Distributions

*It's good to have a sense of things like the scale and the spread of the data you're working with. But looking at the distribution of each feature can be a helpful way to determine if more features are needed.*

*Look for outliers. Take a closer look at the observations that stand out from the rest. Is there anything about these data points that might explain why they aren't behaving like the others? If so, that's a feature you would want to add to your model, to control for such a big difference.*

*Look for clusters in your histograms and scatterplots of various features. If the variance is inconsistent, with dense clusters of observations within the feature space, try to figure out what makes those observations so much more similar than the others. This could point to another feature to add to your data. Investigate any unusual patterns that look like a feature is not being drawn from a random distribution.*

*Look for multiple modes within the distributions of your features. This is a sign that there are categorical variables within your data set. Try to find out what these classes are, if you don't already know, and label your observations if possible.*

### Explore Relationships

*The "eyeball test" is the quickest way to check that your hypothesis is on the right track. Plot your inputs against your outputs. If there is real predictive power for the model you are trying to build, you should be able to make reasonable guesses (just by pointing at the chart, no need for precise values yet) of what the output should be for any given input value.*

*In a regression model, we are looking for clear relationships between our features and targets. Generate pairplots and look for features that have high correlations with the target variable.*

*In a classification model, we are looking for features that separate the population into distinct distributions. Generate pairplots that are color-coded by your categories and look for features where the categories have distinct distributions.*

## Model the Data

*Describe the algorithms that you are considering. How do they work? Why are they good choices for this data and problem space?*

*What nuances in the data will you have to be aware of in order to avoid introducing bias to your model? What steps will you need to take to prevent overfitting? What risks are there for data leakage?*

### Train Validation Test Split

*Pay special attention here to what data is going into your training and test sets. Is there any data leakage? Make sure that you are testing your model on information it has not seen during the learning process. If this is a classification problem, make sure that your classes are reasonably balanced.*

*Also, keep in mind that after you've trained and evaluated your model, you may want to look at the results of specific missed predictions. Set up your training and test data sets so that it will be easy to identify the record id when you are looking at a given prediction result.*

### Preprocessing

*These are standard processing steps to prepare your data for modeling. But remember that any code you use to scale or encode your data has to come from the observations in your training set.*

#### Feature Scaling

#### Label Encoding

### Build and Train Model

*Write down any thoughts you may have about working with these algorithms on this data. What looks to have been the most successful design choices? What pain points are you running into? What other ideas do you want to try out as you iterate on this pipeline?*

### Predict and Score

Important: Stick to the metrics you chose to test before you trained and evaluated your model! Remember that this is the standard you selected before your analysis as the one that you would find most convincing that the model works.

## Interpret the Model

Write up the things you learned, and how well your model performed. Be sure address the model's strengths and weaknesses. What types of data does it handle well? What types of observations tend to give it a hard time? What future work would you or someone reading this might want to do, building on the lessons learned and tools developed in this project?

#### Report Metrics in Context
How did the model perform on the key metrics you chose to demonstrate its usefulness? What counts as good or bad performance? How well do humans perform on this task? How well would you expect to do with random dice rolls? What are the costs associated with missed predictions?

#### Inspect Errors

Look at some of the observations with missed predictions. (Do this in your validation set, never look at individual records in your test set!) Are there common patterns among the observations with bad results? Do you have data that you can include in your model that will capture these patterns or is this a task that will need another research project to solve?

This is your last step in the iteration cycle; if you can't find anything else you can work on here with your present data, project scope, and deadlines, then it's time to wrap things up.

#### Strengths and Weaknesses

Once you've gone through your iteration cycles and are finished with this version of the model, or this particular project, provide an assessment of what types of observations are handled well by your model, and what circumstances seem to give it trouble. This will point you and others towards more questions for future projects.

## Next Steps: What Can We Do Now?

Reporting a model's results is good, and is the main objective of any data science project. But a project is one thing, a career is another. A question is one thing, but science is another. If you've carried out your research with a mindset of curiosity and creativity, then by now you should have plenty more, and much better informed, questions about this topic than what you started with.

So in addition to reporting on the question you investigated and the answers you found, think of the needs of your team, your users, and your peers in the industry, and make some recommendations that answer these two questions:

What are some unanswered questions in my project where more information (additional data sources, deeper understanding, other models or tools) might help improve these results?
What are other needs or problems where my model or my approach may be useful?

In [15]:
from folium import plugins
import folium


# inspired by https://alysivji.github.io/getting-started-with-folium.html
def map_points(df, lat_col='latitude', lon_col='longitude', zoom_start=11, \
                plot_points=False, pt_radius=15, \
                draw_heatmap=False, heat_map_weights_col=None, \
                heat_map_weights_normalize=True, heat_map_radius=15):
    """Creates a map given a dataframe of points. Can also produce a heatmap overlay

    Arg:
        df: dataframe containing points to maps
        lat_col: Column containing latitude (string)
        lon_col: Column containing longitude (string)
        zoom_start: Integer representing the initial zoom of the map
        plot_points: Add points to map (boolean)
        pt_radius: Size of each point
        draw_heatmap: Add heatmap to map (boolean)
        heat_map_weights_col: Column containing heatmap weights
        heat_map_weights_normalize: Normalize heatmap weights (boolean)
        heat_map_radius: Size of heatmap point

    Returns:
        folium map object
    """

    ## center map in the middle of points center in
    middle_lat = df[lat_col].median()
    middle_lon = df[lon_col].median()

    curr_map = folium.Map(location=[middle_lat, middle_lon],
                          zoom_start=zoom_start)

    # add points to map
    if plot_points:
        for _, row in df.iterrows():
            folium.CircleMarker([row[lat_col], row[lon_col]],
                                radius=pt_radius,
                                popup=row['location'],
                                fill_color="#3db7e4", # divvy color
                               ).add_to(curr_map)

    # add heatmap
    if draw_heatmap:
        # convert to (n, 2) or (n, 3) matrix format
        if heat_map_weights_col is None:
            cols_to_pull = [lat_col, lon_col]
        else:
            # if we have to normalize
            if heat_map_weights_normalize:
                df[heat_map_weights_col] = \
                    df[heat_map_weights_col] / df[heat_map_weights_col].sum()

            cols_to_pull = [lat_col, lon_col, heat_map_weights_col]

        stations = df[cols_to_pull].values
        curr_map.add_child(plugins.HeatMap(stations, radius=heat_map_radius))

    return curr_map

In [17]:
map_points(
    df=df[
        df 
        & ~df.lat.isnull()
    ].sample(500, random_state=911) , # for memory constrains
    lat_col='lat',
    lon_col='lon', 
    zoom_start=2, # change to `1` to set global world view
    plot_points=False, 
    pt_radius=0.75,
    draw_heatmap=True, 
    heat_map_weights_col='target',
    heat_map_weights_normalize=True,
    heat_map_radius=15,
)

AttributeError: 'DataFrame' object has no attribute 'lat'